In [2]:
import numpy as np
import struct


def loadImage(file_name):
    file= open(file_name, 'rb')  # read only
    f = file.read()

    magic, imgNum, row, col = struct.unpack_from('>IIII' , f ,0)
    
    offset = struct.calcsize('>IIII')
    bits = imgNum * row * col
    bitsString = '>' + str(bits) + 'B'
    imgs = list(struct.unpack_from(bitsString,f,offset))

    file.close()
    
    for i in range(bits):
        if imgs[i] < 50:
            imgs[i] = 0
        else:
            imgs[i] = 1
    
    imgs = np.array(imgs)
    imgs = imgs.reshape(imgNum, row*col)
    
    return imgs  # matrix, imgNum * pixel


def loadLabel(file_name):
    file = open(file_name, 'rb')  # read only
    f = file.read()

    magic, lablNum = struct.unpack_from('>II' , f ,0)

    offset = struct.calcsize('>II')
    numString = '>' + str(lablNum) + "B"
    labels = list(struct.unpack_from(numString, f, offset))
    
    file.close()

    # labels = np.reshape(labels,[imgNum,1])
    return labels  # list

def loadData():
    imgsTrain = loadImage("train-images.idx3-ubyte")
    labelsTrain = loadLabel("train-labels.idx1-ubyte")
    imgsTest = loadImage("t10k-images.idx3-ubyte")
    labelsTest = loadLabel("t10k-labels.idx1-ubyte")
    
    print("load all data set finished")
    return imgsTrain, labelsTrain, imgsTest, labelsTest
 
if __name__=="__main__":
loadData()

IndentationError: ignored

In [1]:
import time
import numpy as np
#import pandas as pd
from collections import Counter
from loadDataSet import loadData


class treeNode:
    def __init__(self, label = None, feature = None):
        self.child = {} # dict, key is 0 or 1, value is subtree(treeNode)
        self.label = label  # if leaf node, it is the label
        self.feature = feature # feature have largest info gain ratio

    def add_tree(self, key, subtree):
        self.child[key] = subtree

    def predict(self, features): 
        if len(self.child) == 0 or (features[self.feature] not in self.child):
            return self.label
        tree = self.child.get(features[self.feature])
        return tree.predict(features)


def entropy(x):
    values = np.unique(x)  # x is an array, shape[0] means amounts
    ans = 0.0
    for value in values:
        p = float(x[x == value].shape[0]) / x.shape[0]
        log_p = np.log2(p)
        ans -= p * log_p
    return ans


def gainInfoRatio(x, y):
    values = np.unique(x)
    cond = 0.0
    for value in values:
        sub_y = y[x == value]
        ent = entropy(sub_y)
        cond += (float(sub_y.shape[0]) / y.shape[0]) * ent
    ans = entropy(y) - cond
    return ans/entropy(y)


def DT(train_images, train_labels, features):  # return a tree
    # pure case
    if len(set(train_labels)) == 1:
        return treeNode(label = train_labels[0])

    c = Counter(np.array(train_labels))
    common = c.most_common()[0][0]  # the most common label
   
    if len(features) == 0:  # no more features to split
        return treeNode(label = common)
   
    best_feature = 0  # choose the best feature to split
    max_gain = 0
    
    for feature in features:
        GIR = gainInfoRatio(train_images[:, feature], np.array(train_labels))
        if GIR > max_gain:
            max_gain = GIR
            best_feature = feature

    if max_gain < 0.005:  # pre_trim to avoid overfitting
        return treeNode(label = common)

    #build subtree
    sub_features = list(filter(lambda x : x != best_feature, features))  # consider features left
    tree = treeNode(feature = best_feature)

    #max_feature_col = np.array(train_set[:,max_feature].flat)
    values = np.unique(train_images[:, best_feature])  # the values of best_featuer
    for value in values:  # value can only be 0 or 1
        onekind = []
        for i in range(len(train_labels)):
            if train_images[i][best_feature] == value:  # the ith image
                onekind.append(i)
        arr_train_images = np.array(train_images)  
        arr_train_labels = np.array(train_labels)
        sub_tree = DT(arr_train_images[onekind], arr_train_labels[onekind] ,sub_features)
        tree.add_tree(value, sub_tree)

    return tree


def predict(test_images, tree):
    prediction = []
    for each in test_images:  # each image is a row
        each_predict = tree.predict(each)
        prediction.append(each_predict)
    return prediction  # a list


def Accuracy(predictions, test_labels):  # both parameters are lists
    correct = 0
    for i in range(len(predictions)):
        if predictions[i] == test_labels[i]:
            correct += 1
    return correct/len(test_labels)

def main():
    train_images, train_labels, test_images, test_labels = loadData()
    start = time.time()
    decision_tree = DT(train_images, train_labels, list(range(784)))  # 28 * 28 features
    prediction = predict(test_images, decision_tree)
    end = time.time()
    
    print('Accuracy:', Accuracy(prediction, test_labels))
    print('DecisionTree time;', str(end-start), 's')
    
    
if __name__ == "__main__":
main()

IndentationError: ignored